| [02_advanced/06_装饰器.ipynb](https://github.com/shibing624/python-tutorial/blob/master/02_advanced/06_装饰器.ipynb)  | Decorator装饰器  |[Open In Colab](https://colab.research.google.com/github/shibing624/python-tutorial/blob/master/02_advanced/06_装饰器.ipynb) |

# 装饰器：Decorator
如果你有一批变量想统一按一个规则处理，并且需要缩减代码，你需要函数。

如果你有一批函数想统一按一个规则处理，并且需要缩减代码，你需要装饰器（Decorator）

理清下面2点：

函数
- 接受参数
- 做点事情
- 返回结果


装饰器
- 接受函数作为参数
- 做点事情
- 返回一个函数


用 @ 来使用装饰器

使用 @ 符号来将某个函数替换为装饰符之后的函数：

例如这个函数：

In [5]:
def dec(f):
    print('I am decorating function', id(f))
    return f

In [6]:
def foo(x):
    print(x)  # I am decorating function 45206384

In [7]:
foo = dec(foo)

I am decorating function 1627318192304


可以替换为：

In [28]:
@dec
def foo(x):
    print(x)

I am decorating function 140410366082496


### 例子
定义两个装饰器函数，一个将原来的函数值加一，另一个乘二：

In [29]:
def plus_one(f):
    def new_func(x):
        return f(x) + 1

    return new_func

In [30]:
def times_two(f):
    def new_func(x):
        return f(x) * 2

    return new_func

定义函数，先乘二再加一：

In [31]:
@plus_one
@times_two
def foo(x):
    return int(x)

In [32]:
b = foo(2)
b  # 5

5

## 修饰器工厂
decorators factories 是返回修饰器的函数

它的作用在于产生一个可以接受参数的修饰器，

例如我们想将 函数 输出的内容写入一个文件去，可以这样做：

In [8]:
def super_loud(filename):
    fp = open(filename, 'w')

    def loud(f):
        def new_func(*args, **kw):
            fp.write(str(args))
            fp.writelines('\n')
            fp.write('calling with' + str(args) + str(kw))
            # 确保内容被写入
            fp.flush()
            fp.close()
            rtn = f(*args, **kw)
            return rtn

        return new_func

    return loud

In [9]:
@super_loud('test.txt')
def foo(x):
    print(x)


# 调用 foo 就会在文件中写入内容：
foo(100)

100


In [45]:
import os
os.remove('test.txt')

## @classmethod 装饰器
在 Python 标准库中，有很多自带的装饰器，

例如 classmethod 将一个对象方法转换了类方法：

In [35]:
class Foo(object):
    @classmethod
    def bar(cls, x):
        print('the input is', x)

    def __init__(self):
        pass

类方法可以通过 类名.方法 来调用：

In [36]:
Foo.bar(10)

the input is 10


## @property 装饰器
有时候，我们希望像 Java 一样支持 getters 和 setters 的方法，

这时候就可以使用 property 装饰器：

In [37]:
class Foo(object):
    def __init__(self, data):
        self.data = data

    @property
    def x(self):
        return self.data


此时可以使用 .x 这个属性查看数据（不需要加上括号）：

In [38]:
foo = Foo(22)
print(foo.x)

22


这样做的好处在于，这个属性是只读的：

foo.x = 1 会报错


如果想让它变成可读写，可以加上一个装饰符 @x.setter：

In [11]:
class Foo(object):
    def __init__(self, data):
        self.data = data

    @property
    def x(self):
        return self.data

    @x.setter
    def x(self, value):
        self.data = value

NameError: name 'x' is not defined

In [40]:
foo = Foo(1000)
foo.x

1000

In [42]:
foo.x = 2222
foo.x

2222

## 应用：定时器

要求：写一个定时器功能，要求监控一个执行程序，超时则报警。

如何完成？

In [55]:

import signal
import time

# very nice function
def set_timeout(num, callback):
    def wrap(func):
        def handle(signum, frame):  # 收到信号 SIGALRM 后的回调函数，参数1是信号的数字，参数2是the interrupted stack frame.
            raise RuntimeError

        def to_do(*args, **kwargs):
            try:
                signal.signal(signal.SIGALRM, handle)  # 设置信号和回调函数
                signal.alarm(num)  # 设置 num 秒的闹钟
                print('start alarm signal.')
                r = func(*args, **kwargs)
                print('close alarm signal.')
                signal.alarm(0)  # 关闭闹钟
                return r
            except RuntimeError as e:
                callback()

        return to_do

    return wrap


def after_timeout():  # 超时后的处理函数
    print("do something after timeout.")
    raise RuntimeError


@set_timeout(2, after_timeout)  # 限时 2 秒超时
def connect():  # 要执行的函数
    time.sleep(2.4)  # 函数执行时间，写大于2的值，可测试超时
    return "完成"

class Demo:
    @set_timeout(2, after_timeout)
    def conn(self):
        time.sleep(3)
        return "ok"

试一下：

In [56]:
try:
    a = connect()
    print(a)
except Exception as e:
    a = 'err'
    print(a)


start alarm signal.
do something after timeout.
err


In [58]:
b = Demo()
try:
    c = b.conn()
    print(c)
except RuntimeError as e:
    print('run time err.')

start alarm signal.
close alarm signal.
ok


如果不超时：

In [59]:
class Demo:
    @set_timeout(2, after_timeout)
    def conn(self):
        time.sleep(1)
        return "ok"
    
b = Demo()
try:
    c = b.conn()
    print(c)
except RuntimeError as e:
    print('run time err.')

start alarm signal.
close alarm signal.
ok


本节完。

In [2]:
def hi(name='yasoob'):
    return "hi " + name

print(hi())
# output: "hi yasoob"

# 我们甚至可以将一个函数赋值给一个变量，比如
greet = hi

# 我们这里没有在使用小括号，因为我们并不是在调用hi函数
# 而是在将它放在greet变量里头

print(greet())

def hi(name="yasoob"):
    print("now you are inside the hi() function")
    
    def greet():
        return "now you are in the greet() function"
    
    def welcome():
        return "now you are in the welcome() function"
    
    print(greet())
    print(welcome())
    print("now you are back in the hi() function")

hi()

hi yasoob
hi yasoob
now you are inside the hi() function
now you are in the greet() function
now you are in the welcome() function
now you are back in the hi() function


In [4]:
def hi(name="yasoob"):
    def greet():
        return "now you are in the greet() function"

    def welcome():
        return "now you are in the welcome() function"
    
    if name == 'yasoob':
        return greet
    else:
        return welcome
a = hi()
print(a)
print(a())

<function hi.<locals>.greet at 0x0000025FC8AC29D0>
now you are in the greet() function


In [7]:
def hi():
    return "hi yasoob"

def doSomethingBeforeHi(func):
    print("I am doing some boring work before executing hi()")
    print(func())
    
doSomethingBeforeHi(hi)

I am doing some boring work before executing hi()
hi yasoob


In [12]:
def a_new_decorator(a_func):
    
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        
        a_func()
        
        print("I am doing some boring work after executing a_func()")
        
    return wrapTheFunction

@a_new_decorator
def a_function_requiring_decoration():
    print("I am the function which needs some decoration to remove my foul smell")

    
a_function_requiring_decoration()

# a_function_requiring_decoration = a_new_decorator(a_function_requiring_decoration)
# a_function_requiring_decoration()
print(a_function_requiring_decoration.__name__)

I am doing some boring work before executing a_func()
I am the function which needs some decoration to remove my foul smell
I am doing some boring work after executing a_func()
wrapTheFunction


In [13]:
from functools import wraps

def a_new_decorator(a_func):
    @wraps(a_func)
    def wrapTheFunction():
        print("I am doing some boring work before executing a_func()")
        a_func()
        print("I am doing some boring work after executing a_func()")
    return wrapTheFunction

@a_new_decorator
def a_function_requiring_decoration():
    """Hey yo! Decorate me"""
    print("I am the function which needs some decoration to remove my foul smell")
    
print(a_function_requiring_decoration.__name__)

a_function_requiring_decoration


In [17]:
from functools import wraps

def decorator_name(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        if not can_run:
            return "Function will not run"
        return f(*args, **kwargs)
    return decorated


@decorator_name
def func():
    return("Function is running")

can_run = True
print(func())

Function is running


In [18]:
from functools import wraps

def requires_auth(f):
    @wraps(f)
    def decorated(*args, **kwargs):
        auth = request.authorization
        if not auth or not checkauth(auth.name, auth.password):
            authenticate()
        return f(*args, **kwargs)
    return decorated


KeyboardInterrupt: 

In [20]:
from functools import wraps

def logits(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + "was called")
        return func(*args, **kwargs)
    return with_logging

@logits
def addition_func(x):
    """Do some math."""
    return x + x

In [25]:
from functools import wraps

def logit(logfile='out.log'):
    def logging_decorator(func):
        @wraps(func)
        def wrapped_function(*args, **kwargs):
            log_string = func.__name__ + "was called"
            print(log_string)
            # 打开log_file,并写入内容
            with open(logfile, "a") as opened_file:
                # 现在将日志打到指定的logfile
                opened_file.write(log_string + '\n')
            return func(*args, **kwargs)
        return wrapped_function
    return logging_decorator

@logit()
def myfunc1():
    pass

myfunc1()

myfunc1was called


In [32]:
class logit(object):
    _logfile = 'out.log'
    
    def __init__(self, func):
        self.func = func
    
    def __call__(self, *args):
        log_string = self.func.__name__ + "was called"
        print(log_string)
        # 打开logfile写入
        with open(self._logfile, "a") as opened_file:
            # 现实将日志打到指定的文件
            opened_file.write(log_string + '\n')
        # 现在，发送一个通知
        self.notify()
        
        # return base func
        return self.func(*args)

    def notify(self):
        # logit只打日志，不做别的
        pass
    
logit._logfile = 'out2.log'
@logit
def myfunc1():
    pass

myfunc1()

class email_logit(logit):
    
    '''一个logit的实现版本，可以在函数调用时发送email给管理员'''
    def __init__(self, email='admin@myproject.com', *arg, **kwargs):
        
       self.email = email
       super(email_logit, self).__init__(*args, *kwargs)
    
    def notify(self):
        # 发送一封email到self.email
        pass 

myfunc1was called


In [35]:
# python中可变数据类型有:字典dict, 列表list, 集合set;
# 不可变数据类型有:整型int,浮点型float,字符串string和元组tuple
from collections import defaultdict

colours = (
    ('Yasoob', 'Yellow'),
    ('Ali', 'Blue'),
    ('Arham', 'Green'),
    ('Ali', 'Black'),
    ('Yasoob', 'Red'),
    ('Ahmed', 'Silver'),
)

favourite_colours = defaultdict(list)

for name, colour in colours:
    favourite_colours[name].append(colour)

print(favourite_colours)
some_dict = {}
some_dict['colours']['favourite'] = "yellow"


defaultdict(<class 'list'>, {'Yasoob': ['Yellow', 'Red'], 'Ali': ['Blue', 'Black'], 'Arham': ['Green'], 'Ahmed': ['Silver']})


KeyError: 'colours'

In [40]:
import collections
import json
tree = lambda: collections.defaultdict(tree)
some_dict = tree()
some_dict['colours']['favourite']['level1'] = "yellow"
print(json.dumps(some_dict))
## 运行正常

{"colours": {"favourite": {"level1": "yellow"}}}


In [42]:
from collections import Counter

colours = (
    ('Yasoob', 'Yellow'),
    ('Ali', 'Blue'),
    ('Arham', 'Green'),
    ('Ali', 'Black'),
    ('Yasoob', 'Red'),
    ('Ahmed', 'Silver')
)

favs = Counter(name for name, colour in colours)
print(favs)

Counter({'Yasoob': 2, 'Ali': 2, 'Arham': 1, 'Ahmed': 1})


In [46]:
from collections import namedtuple

Animal = namedtuple('Animal', 'name age types')
perry = Animal(name='perry', age=31, types="cat")

print(perry)

print(perry.name)

Animal(name='perry', age=31, types='cat')
perry


In [ ]:
from collections import namedtuple
from enum import Enum

class Species(Enum):
    cat = 1
    dog = 2
    hose = 3
Animal = namedtuple('Animal', 'name age type')
perry = Animal(name="Perry", age=31, type=Species.cat)
drogon = Animal(name="Drogon", age=4, type=Species.dragon)
tom = Animal(name="Tom", age=75, type=Species.cat)
charlie = Animal(name="Charlie", age=2, type=Species.kitten)    

In [1]:
from collections import namedtuple
from enum import Enum

class Species(Enum):
    cat = 1
    dog = 2
    horse = 3
    aardvark = 4
    butterfly = 5
    owl = 6
    platypus = 7
    dragon  = 8
    unicorn = 9
    
    kitten = 1
    puppy = 2
    
Animal = namedtuple("Animal", 'name age type')
perry = Animal(name='Perry', age=31, type=Species.cat)
dragon = Animal(name="Perry", age = 31, type = Species.dragon)
tom = Animal(name="Tom", age=75, type=Species.cat)
charlie = Animal(name="Charlie", age=2, type=Species.kitten)

In [5]:
# 列表推导式
# variable = [out_exp for out_exp in input_list if out_exp == 2]

multiples = [i for i in range(30) if i % 3 == 0]
print(multiples)
squared = []
for x in range(10):
    squared.append(x ** 2)

[0, 3, 6, 9, 12, 15, 18, 21, 24, 27]


In [6]:
mcase = {'a':10, 'b': 34, 'A': 7, 'Z': 3}
mcase_frequency = {
    k.lower(): mcase.get(k.lower(), 0) + mcase.get(k.upper(), 0)
    for k in mcase.keys()
}

In [8]:
grades = {"Alice":90, "Bob":85, "Charles":92}
stella_grade = grades.get("stella", "she is a stupid")
print(stella_grade)

she is a stupid
